In [1]:
print("hello world")

hello world


In [24]:
import os
import sys
import traceback
import pandas as pd
import numpy as np
import re
import json
import requests
import subprocess

def get_source_content(url):
    response = requests.get(url)

    if response.status_code == 200:
        return response.content
    else:
        return None
    

def get_diff_num(filename):
    diff_start_lines = []
    with open(filename, "r") as patch:
        for i, line in enumerate(patch):
            if line.startswith("@@ "):
                if not i == 0:
                    diff_start_lines.append(i)
        diff_start_lines.append(i+1)
    return diff_start_lines


def get_enumerate(filename):
    patch = open(filename, "r")
    return enumerate(patch)


def get_diff_information(filename, diff_start_lines):
    block_num = 0
    archor = []
    count = 0
    for diff_start_line in diff_start_lines:
        # reset some values
        count += 1
        minus_count = 0
        plus_count = 0
        before = None
        after = None
        start_line_num = None
        minus_pos = []
        plus_pos = []
        patch = []
        for j, l in get_enumerate(filename):
            if count == len(diff_start_lines):
                if j == diff_start_line - 1:
                    patch.append(l)
                    if l.startswith("-"):
                        minus_count += 1
                        minus_pos.append(j - start_line_num)
                    if l.startswith("+"):
                        plus_count += 1
                        plus_pos.append(j - start_line_num)
                    block_num = j
                    before = list(map(int, before))
                    after = list(map(int, after))
                    if len(archor) == 0:
                        archor.append(
                            [before, after, minus_count, plus_count, minus_pos, plus_pos, patch, after[0], minus_count,
                             0])
                    else:
                        diff_value = archor[-1][8] + minus_count
                        insert_after = after[0] + archor[-1][8]
                        last_end = archor[-1][1][0] + archor[-1][1][1] - 1
                        archor.append([before, after, minus_count, plus_count, minus_pos, plus_pos, patch, insert_after,
                                       diff_value, last_end])
                    break

                if block_num <= j < diff_start_line - 1:
                    patch.append(l)
                    if l.startswith("@@ "):
                        start_line_num = j
                        pos = l.find("@@ ")
                        end = l.find(" @@ ")
                        modified = l[pos + 3:end]
                        modified = modified.split(" ")
                        before = modified[0]
                        before = before.replace("-", "")
                        before = before.split(",")
                        after = modified[1]
                        after = after.replace("+", "")
                        after = after.split(",")
                        # now our source files are after-modified

                    if l.startswith("-"):
                        minus_count += 1
                        minus_pos.append(j - start_line_num)

                    if l.startswith("+"):
                        plus_count += 1
                        plus_pos.append(j - start_line_num)
                elif j < block_num:
                    continue
                else:
                    block_num = j
                    before = list(map(int, before))
                    after = list(map(int, after))
                    if len(archor) == 0:
                        archor.append(
                            [before, after, minus_count, plus_count, minus_pos, plus_pos, patch, after[0], minus_count,
                             0])
                    else:
                        diff_value = archor[-1][8] + minus_count
                        insert_after = after[0] + archor[-1][8]
                        last_end = archor[-1][1][0] + archor[-1][1][1] - 1
                        archor.append([before, after, minus_count, plus_count, minus_pos, plus_pos, patch, insert_after,
                                       diff_value, last_end])
                    break
            else:
                if block_num <= j < diff_start_line:
                    patch.append(l)
                    if l.startswith("@@ "):
                        start_line_num = j
                        pos = l.find("@@ ")
                        end = l.find(" @@ ")
                        modified = l[pos + 3:end]
                        modified = modified.split(" ")
                        before = modified[0]
                        before = before.replace("-", "")
                        before = before.split(",")
                        after = modified[1]
                        after = after.replace("+", "")
                        after = after.split(",")
                        # now our source files are after-modified

                    if l.startswith("-"):
                        minus_count += 1
                        minus_pos.append(j - start_line_num)

                    if l.startswith("+"):
                        plus_count += 1
                        plus_pos.append(j - start_line_num)
                elif j < block_num:
                    continue
                else:
                    block_num = j
                    before = list(map(int, before))
                    after = list(map(int, after))
                    if len(archor) == 0:
                        archor.append(
                            [before, after, minus_count, plus_count, minus_pos, plus_pos, patch, after[0], minus_count,
                             0])
                    else:
                        diff_value = archor[-1][8] + minus_count
                        insert_after = after[0] + archor[-1][8]
                        last_end = archor[-1][1][0] + archor[-1][1][1] - 1
                        archor.append([before, after, minus_count, plus_count, minus_pos, plus_pos, patch, insert_after,
                                       diff_value, last_end])
                    break

    return archor

def get_line_numbers(filename,lang_type):
    # found = False
    #cmd = "ctags -x --c-kinds=fp " + filename + " | grep " + funcname
    cmd = "ctags -x --"+lang_type+"-kinds=f " + filename

    output = subprocess.getoutput(cmd)
    lines = output.splitlines()
    line_nums = []
    for line in lines:
        line = line.split(" ")
        char = list(filter(None, line))
        line_num = char[2]
        line_nums.append(int(line_num))
    return line_nums


def generate_line_diff(c_cpp_csv):
    vul_number = 0 
    lname=[]
    vul_functions_before = []
    vul_functions_after = []
    data=[]

    for index, row in c_cpp_csv.iterrows():
        try:
            commit_id = row["commit_id"]
            # print("this is commit id",commit_id)
            diff = row["files_changed"]
            if not (row["cwe_id"] is None) and not (row["cwe_id"] == ""):
                CWE_ID = str(row["cwe_id"])
            else:
                CWE_ID = "others"
            files_changed = []
            project = row["project"]
            for i in diff.split("<_**next**_>"):
                files_changed.append(json.loads(i))
            for file in files_changed:
                file_with_dir = file["filename"]
                # print("file_with_dir", file_with_dir)
                pos = file_with_dir.rfind('/')
                if pos > 0:
                    filename = file_with_dir[pos + 1:]
                    file_dir = commit_id + "/" + file_with_dir[:pos]
                elif pos == 0:
                    filename = file_with_dir[1:]
                    file_dir = commit_id
                else:
                    filename = file_with_dir
                    file_dir = commit_id
                raw_url=file["raw_url"]
                print("raw_url",raw_url)
                if "patch" in file:
                    patch = file["patch"]
                else:
                    patch = ""
                type_pos = filename.find('.')
                if type_pos > 0:
                    only_name = filename[:type_pos]
                    # print("only_name", only_name)
                    only_type = filename[type_pos + 1:]
                    # print("only_type", only_type)

                else:
                    only_name = filename
                    only_type = "not know"
                
                sourcefiles = get_source_content(raw_url)
                if sourcefiles is not None:
                   sourcefiles_str = sourcefiles.decode('utf-8') 
                # TODO: get sourcefiles from local
                if not os.path.exists("patches/" + only_type + '/' + project + '/' + CWE_ID + '/' + file_dir):
                    os.makedirs("patches/" + only_type + '/' + project + '/' + CWE_ID + '/' + file_dir)
                sourcefile_dir = "patches/" + only_type + '/' + project + '/' + CWE_ID + '/' + file_dir + '/' + filename
                # print(sourcefile_dir)
                patchfile_dir = "patches/" + only_type + '/' + project + '/' + CWE_ID + '/' + file_dir + '/' + only_name + '_' + 'patch.txt'
                # print(patchfile_dir)
                with open(sourcefile_dir, "w+") as source_file, open(patchfile_dir, "w+") as patch_file:
                    source_file.write(sourcefiles_str)
                    patch_file.write(patch)
                
                 # get functions: if vul? not vul?
                if only_type == "c":
                    num = get_diff_num(patchfile_dir)
                    print("num", num)
                    archors = get_diff_information(patchfile_dir, num)
                    print("archors", archors)
                    block_num = 0
                    block_total = len(archors)
                    for archor in archors:
                        block_num += 1
                        
                        # data=[]
                        del_line_pos = archor[4]
                        add_line_pos = archor[5]
                        patch_start = int(archor[1][0])
                        # print("patch_start", patch_start)
                        patch_lines = int(archor[0][1]) + archor[3]
                        # print("patch_lines", patch_lines)
                        patch_end = patch_start + patch_lines - 1
                        # print("patch_end", patch_end)
                        source_end = patch_start + int(archor[1][1]) - 1
                        # print("source_end", source_end)
                        last_end = archor[9]
                        # print("last_end", last_end)
                        wrote = False
                        add_patch_file_dir = "patches/" + only_type + '/' + project + '/' + CWE_ID + '/' + file_dir + '/' + "add_patch_" + filename
                        print("add_patch_file_dir", add_patch_file_dir)
                        with open(sourcefile_dir, "r") as before, open(add_patch_file_dir, "a") as after:
                            lines = before.readlines()
                            # print("lines", lines)
                            flen = len(lines)
                            # print("flen", flen)
                            for i in range(flen):
                                if last_end - 1 < i <= source_end - 1:
                                    if i == 0:
                                        after.write(lines[i])
                                        continue
                                    if (patch_start - 1 <= i <= source_end - 1):
                                        if wrote == False:
                                            for patch_line in archor[6][1:]:
                                                if patch_line.startswith("+"):
                                                    patch_line = patch_line.replace("+", "//fix_flaw_line_below:\n//",
                                                                                     1)
                                                if patch_line.startswith("-"):
                                                    patch_line = patch_line.replace("-", "//flaw_line_below:\n", 1)
                                                if not patch_line.endswith("\n"):
                                                    patch_line = patch_line + "\n"
                                                after.write(patch_line)
                                            wrote = True
                                    else:
                                        after.write(lines[i])
                                if block_num == block_total and source_end < flen and i > source_end - 1:
                                    after.write(lines[i])

                            line_nums = get_line_numbers(add_patch_file_dir, "c")
                            print("line_nums", line_nums)    
        except Exception as e:
            traceback.print_exc(file=sys.stdout)
            print("reason", e)
            print("\n commit_id:" + str(commit_id) + "！")
            print("\n index:" + str(index) + "！")
            continue


if __name__ == '__main__':
    c_cpp_csv = pd.read_csv('sample_cpp.csv', nrows=1,encoding='utf-8')
    result=generate_line_diff(c_cpp_csv)

raw_url https://github.com/bratsche/pango/raw/4de30e5500eaeb49f4bf0b7a07f718e149a2ed5e/pango/glyphstring.c
num [35]
archors [[[61, 14], [61, 28], 6, 20, [4, 9, 10, 11, 13, 14], [5, 6, 7, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31], ['@@ -61,14 +61,28 @@ pango_glyph_string_set_size (PangoGlyphString *string, gint new_len)\n', '   while (new_len > string->space)\n', '     {\n', '       if (string->space == 0)\n', '-\tstring->space = 1;\n', '+\t{\n', '+\t  string->space = 4;\n', '+\t}\n', '       else\n', '-\tstring->space *= 2;\n', '-\n', '-      if (string->space < 0)\n', ' \t{\n', '-\t  g_warning ("glyph string length overflows maximum integer size, truncated");\n', '-\t  new_len = string->space = G_MAXINT - 8;\n', '+\t  const guint max_space =\n', '+\t    MIN (G_MAXINT, G_MAXSIZE / MAX (sizeof(PangoGlyphInfo), sizeof(gint)));\n', '+\n', '+\t  guint more_space = (guint)string->space * 2;\n', '+\n', '+\t  if (more_space > max_space)\n', '+\t    {\n', '+\t      mo